In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 574.9MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6121e000 @  0x7f69c129c2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [2]:
!git clone https://github.com/ceshine/examples.git

Cloning into 'examples'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1719 (delta 2), reused 6 (delta 2), pack-reused 1711
Receiving objects: 100% (1719/1719), 38.82 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (904/904), done.


In [3]:
!nvidia-smi

Fri Oct 12 00:17:01 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    85W / 149W |      0MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Train a Language Model

In [5]:
%cd examples/word_language_model/

/content/examples/word_language_model


In [15]:
%%time 
!python -u main.py --cuda --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --tied  2>&1 | tee train.log  # Test perplexity of 75.96

Number of tokens:
Train:  2075677
Valid:  216347
Test:   244102
| epoch   1 |   200/ 2965 batches | lr 20.00 | ms/batch 114.60 | loss  7.73 | ppl  2281.15
| epoch   1 |   400/ 2965 batches | lr 20.00 | ms/batch 113.97 | loss  6.82 | ppl   918.82
| epoch   1 |   600/ 2965 batches | lr 20.00 | ms/batch 114.35 | loss  6.43 | ppl   618.57
| epoch   1 |   800/ 2965 batches | lr 20.00 | ms/batch 114.58 | loss  6.28 | ppl   533.18
| epoch   1 |  1000/ 2965 batches | lr 20.00 | ms/batch 115.09 | loss  6.15 | ppl   467.23
| epoch   1 |  1200/ 2965 batches | lr 20.00 | ms/batch 115.34 | loss  6.07 | ppl   434.59
| epoch   1 |  1400/ 2965 batches | lr 20.00 | ms/batch 115.57 | loss  5.95 | ppl   384.69
| epoch   1 |  1600/ 2965 batches | lr 20.00 | ms/batch 115.44 | loss  5.95 | ppl   385.32
| epoch   1 |  1800/ 2965 batches | lr 20.00 | ms/batch 115.53 | loss  5.81 | ppl   332.43
| epoch   1 |  2000/ 2965 batches | lr 20.00 | ms/batch 115.83 | loss  5.76 | ppl   318.61
| epoch   1 |  2200/ 2965 

### Download the trained model

In [0]:
from google.colab import files
files.download('model.pt')

In [0]:
files.download('train.log')

The above download method doesn't work for me (probably some issues with the proxy). 

Using gsutil here (Only works if you have a Google Cloud Storage account):

In [3]:
%ls -lh pytorch_examples/word_language_model/

total 109M
drwxr-xr-x 3 root root 4.0K Oct 12 05:02 data/
-rw-r--r-- 1 root root 1.5K Oct 12 05:02 data.py
-rw-r--r-- 1 root root 2.6K Oct 12 05:02 generate.py
-rw-r--r-- 1 root root 9.1K Oct 12 05:02 main.py
-rw-r--r-- 1 root root 109M Oct 12 06:52 model.pt
-rw-r--r-- 1 root root 2.5K Oct 12 05:02 model.py
drwxr-xr-x 2 root root 4.0K Oct 12 05:03 __pycache__/
-rw-r--r-- 1 root root 3.2K Oct 12 05:02 README.md
-rw-r--r-- 1 root root    6 Oct 12 05:02 requirements.txt
-rw-r--r-- 1 root root  28K Oct 12 06:54 train.log


In [1]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'personal-project-196600'
!gcloud config set project {project_id}

Updated property [core/project].


In [6]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp model.pt gs://ceshine-colab-tmp/lm_model.pt

Copying file://model.pt [Content-Type=application/octet-stream]...
|
Operation completed over 1 objects/108.5 MiB.                                    
